# Skin Cancer Recognizer

This notebook contains a trained model that, given an image of a sunspot, can detect whether skin cancer is present.

# Define Loaders

Pull in the training/testing/validation images into a loader.

In [59]:
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

# Common parameters/attributes
n_epochs = 50
batch_size = 20
data_dir = 'D:/image_datasets/skin_cancer_images/'
use_cuda = torch.cuda.is_available()

# Data transform will resize and images to 300 x 300, crop out the 
# center at as a 250x250 square and convert the image to a tensor.
image_transform = transforms.Compose([transforms.Resize(256),
                                    transforms.RandomResizedCrop(224), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

# Load in data directories from each relevant folder and transform
sc_data = {
    'train': datasets.ImageFolder(data_dir + 'train', transform=image_transform),
    'valid': datasets.ImageFolder(data_dir + 'valid', transform=image_transform),
    'test': datasets.ImageFolder(data_dir + 'test', transform=image_transform)
}

# Prepare test/train/valid data loaders
sc_loaders = {
    'train': torch.utils.data.DataLoader(sc_data['train'], batch_size=batch_size, num_workers=num_workers, shuffle=True),
    'valid': torch.utils.data.DataLoader(sc_data['valid'], batch_size=batch_size, num_workers=num_workers, shuffle=True),
    'test': torch.utils.data.DataLoader(sc_data['test'], batch_size=batch_size, num_workers=num_workers, shuffle=True)
}

# Image counts
print('Training: ', len(sc_data['train']))
print('Validation: ', len(sc_data['valid']))
print('Testing: ', len(sc_data['test']))

Training:  2000
Validation:  150
Testing:  600


# Define Resnet50 Model

Load in pretrained Resnet50 model and update last FCL for skincare classes

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

## check if CUDA is available
use_cuda = torch.cuda.is_available()


## Add final FCL at the end of the model
sc_model = nn.Sequential(
    models.resnet50(pretrained=True),
    nn.Linear(1000, 3)
)

## move model to GPU if CUDA is available
if use_cuda:
    print('Using cuda')
    sc_model = sc_model.cuda()
else:
    print('Not using cuda')

## Show model structure
print(sc_model)

Using cuda
Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0

)


# Define Optimizer and Criterion

Using SGD optimizer of CrossEntropyLoss criterion

In [61]:
# specify loss function (categorical cross-entropy)
sc_criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.0025
sc_optimizer = optim.SGD(sc_model[1].parameters(), lr=0.0025)

# Define Training Method

A reusable function for training and saving the best model

In [62]:
## When dealing with large images they may be truncated
## and need special permission to load
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model.forward(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
    
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model.forward(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update validation loss
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        # Save model if validation loss has decreased since last min
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [ ]:
## Train the model
sc_model = train(n_epochs, sc_loaders, sc_model, sc_optimizer, sc_criterion, use_cuda, 'cancer_predictor_model.pt')

Epoch: 1 	Training Loss: 0.808634 	Validation Loss: 0.895549
Validation loss decreased (inf --> 0.895549).  Saving model ...
Epoch: 2 	Training Loss: 0.726156 	Validation Loss: 0.820239
Validation loss decreased (0.895549 --> 0.820239).  Saving model ...
Epoch: 3 	Training Loss: 0.702270 	Validation Loss: 0.769043
Validation loss decreased (0.820239 --> 0.769043).  Saving model ...
Epoch: 4 	Training Loss: 0.682880 	Validation Loss: 0.730802
Validation loss decreased (0.769043 --> 0.730802).  Saving model ...
Epoch: 5 	Training Loss: 0.668739 	Validation Loss: 0.756836
Epoch: 6 	Training Loss: 0.663418 	Validation Loss: 0.825605
Epoch: 7 	Training Loss: 0.673556 	Validation Loss: 0.805150
Epoch: 8 	Training Loss: 0.668890 	Validation Loss: 0.716827
Validation loss decreased (0.730802 --> 0.716827).  Saving model ...


# Define Testing Method

Method for testing model performance

In [32]:
def test(loaders, model, criterion, use_cuda):

    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):

        if use_cuda:
            data, target = data.cuda(), target.cuda()

        output = model(data)
        loss = criterion(output, target)
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        pred = output.data.max(1, keepdim=True)[1]
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [33]:
## Test the model
test(sc_loaders, sc_model, sc_criterion, use_cuda)

Test Loss: 0.745554


Test Accuracy: 67% (406/600)


# Predictor Function

Given an image path, it makes a prediction on the type of skin cancer

In [58]:
## Image utils and pt Variable
from PIL import Image
from torch.autograd import Variable

## Prediction method using Resnet50 model
def sc_predict(img_path):
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.RandomResizedCrop(224), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

    image = transform(Image.open(img_path))
    image = image.unsqueeze(0)
    prediction = False

    if use_cuda:
        prediction = sc_model.forward(Variable(image).cuda()).cpu()
    else:
        prediction = sc_model.forward(Variable(image))
    
    return prediction.data.numpy()[0]

print(sc_predict('D:/image_datasets/skin_cancer_images/train/melanoma/ISIC_0000002.jpg'))

[-0.1537728   0.31074837  0.11433992]


# Output to CSV


### Columns:
- Id - the file names of the test images (in the same order as the sample submission file)
- task_1 - the model's predicted probability that the image (at the path in Id) depicts melanoma
- task_2 - the model's predicted probability that the image (at the path in Id) depicts seborrheic keratosis

In [34]:
import csv
import glob
import tqdm

with open('predictions.csv', mode='w') as csv_file:
    ## Define columns
    writer = csv.DictWriter(csv_file, fieldnames=['Id', 'task_1', 'task_2'])
    writer.writeheader()
    
    ## Evaluation mode
    sc_model.eval()
    
    ## Process each file in test directory
    for image_path in tqdm(glob(data_dir + 'test')):
        
        ## Get prediction
        pred = sc_predict(image_path)

        ## Write results to csv
        writer.writerow({
            'Id': image_path, 
            'task_1': pred[0], 
            'task_2': pred[2]
        })

SyntaxError: unexpected EOF while parsing (<ipython-input-34-c0520af0dec7>, line 7)